### Segmentação de Imagens e Avaliação de Qualidade de Segmentação

In [51]:
"""
Módulo para segmentação de imagens utilizando Fuzzy C‑Means (FCM) e K‑Means.

Este módulo reorganiza o código original em funções menores e independentes para
facilitar o reuso e a manipulação dos resultados intermediários. As máscaras de
segmentação (rótulos) são retornadas explicitamente pelas funções de
segmentação. Dessa forma, você pode acessar e manipular diretamente as
mascaras produzidas pelos algoritmos, sem necessidade de percorrer
estruturas complexas.

Funções principais:

* ``load_gray_norm``: carrega uma imagem em escala de cinza e normaliza no
  intervalo ``[0, 1]``.
* ``cmeans_labels``: aplica o algoritmo Fuzzy C‑Means (FCM) e retorna as
  máscaras (labels) e centros.
* ``kmeans_labels``: aplica o algoritmo K‑Means e retorna as máscaras (labels)
  e centros.
* ``run_fcm`` e ``run_kmeans``: funções utilitárias que executam FCM ou
  K‑Means, medindo o tempo de execução e calculando as métricas internas de
  validação de cluster. Cada função retorna um dicionário com as máscaras,
  métricas e tempo de processamento.
* ``evaluate_clustering`` e ``dunn_index``: funções de avaliação que
  calculam índices internos como Davies–Bouldin, Calinski–Harabasz e Dunn.
* ``plot_comparison``: gera uma figura comparando lado a lado as máscaras de
  FCM e K‑Means.
* ``segment_image``: orquestra a segmentação de uma única imagem chamando
  ``run_fcm`` e ``run_kmeans`` e opcionalmente gerando um gráfico de
  comparação.

O objetivo é permitir que cada parte do processamento (carregamento,
segmentação, avaliação e visualização) seja tratada de forma isolada. Isso
facilita o teste e o reaproveitamento das etapas em outros projetos. Se você
deseja acessar diretamente as máscaras de segmentação, elas são retornadas
pelas funções ``run_fcm`` e ``run_kmeans`` nos campos ``"labels"``.
"""

'\nMódulo para segmentação de imagens utilizando Fuzzy\xa0C‑Means (FCM) e K‑Means.\n\nEste módulo reorganiza o código original em funções menores e independentes para\nfacilitar o reuso e a manipulação dos resultados intermediários. As máscaras de\nsegmentação (rótulos) são retornadas explicitamente pelas funções de\nsegmentação. Dessa forma, você pode acessar e manipular diretamente as\nmascaras produzidas pelos algoritmos, sem necessidade de percorrer\nestruturas complexas.\n\nFunções principais:\n\n* ``load_gray_norm``: carrega uma imagem em escala de cinza e normaliza no\n  intervalo ``[0,\xa01]``.\n* ``cmeans_labels``: aplica o algoritmo Fuzzy\xa0C‑Means (FCM) e retorna as\n  máscaras (labels) e centros.\n* ``kmeans_labels``: aplica o algoritmo K‑Means e retorna as máscaras (labels)\n  e centros.\n* ``run_fcm`` e ``run_kmeans``: funções utilitárias que executam FCM ou\n  K‑Means, medindo o tempo de execução e calculando as métricas internas de\n  validação de cluster. Cada função 

## Bibliotecas:

In [52]:
from __future__ import annotations

import time
from pathlib import Path
from typing import Dict, Tuple, List, Any
import matplotlib
matplotlib.use("Agg")
import csv

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from scipy.ndimage import distance_transform_edt
from sklearn.metrics.pairwise import pairwise_distances
from numpy.random import default_rng
from typing import Optional, Any, Dict, Tuple

from docplex.mp.model import Model
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit_optimization.algorithms.admm_optimizer import ADMMOptimizer, ADMMParameters, UPDATE_RHO_BY_RESIDUALS
from qiskit_optimization.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit_optimization.optimizers import COBYLA
#from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization import QuadraticProgram

## Funções Utilitárias (Carregamento e Normalização)

In [53]:
def apply_normalization(
    img: np.ndarray,
    normalize: Optional[str] = None,
    params: Optional[Dict[str, float]] = None,
) -> Tuple[np.ndarray, Optional[Dict[str, float]]]:
    """
    Normaliza opcionalmente a imagem.
    normalize: None | 'minmax' | 'zscore'
    params (opcional): dict com chaves esperadas por método escolhido:
        - 'min','max' para minmax
        - 'mean','std' para zscore
    Retorna (img_norm, used_params)
    """
    if normalize is None:
        return img, None

    if normalize == 'minmax':
        if params is None:
            mn = float(np.min(img))
            mx = float(np.max(img))
        else:
            mn = float(params.get('min', np.min(img)))
            mx = float(params.get('max', np.max(img)))
        denom = (mx - mn) + 1e-10
        out = (img - mn) / denom
        return out, {'method': 'minmax', 'min': mn, 'max': mx}

    if normalize == 'zscore':
        if params is None:
            mu = float(np.mean(img))
            sd = float(np.std(img))
        else:
            mu = float(params.get('mean', np.mean(img)))
            sd = float(params.get('std',  np.std(img)))
        out = (img - mu) / (sd + 1e-10)
        return out, {'method': 'zscore', 'mean': mu, 'std': sd}

    # método desconhecido -> não normaliza
    return img, None




def load_gray_norm(path: Path) -> np.ndarray:
    """Carrega uma imagem em escala de cinza (L) e normaliza para o intervalo [0, 1].

    Parameters
    ----------
    path : Path
        Caminho para a imagem no disco.

    Returns
    -------
    np.ndarray
        Array 2‑D float32 com valores normalizados.
    """
    # Abre a imagem em modo L (8‑bits) e converte para float32
    arr = np.asarray(Image.open(path).convert("L"), dtype=np.float32)
    # Normaliza para [0,1] evitando divisão por zero
    denom = (arr.max() - arr.min()) + 1e-10
    return (arr - arr.min()) / denom


def maybe_flip_binary(pred: np.ndarray, gt: np.ndarray) -> np.ndarray:
    # Ex.: maximiza IoU (poderia ser acurácia, F1, etc.)
    def iou(a, b):
        inter = np.logical_and(a == 1, b == 1).sum()
        union = np.logical_or(a == 1, b == 1).sum()
        return inter / (union + 1e-10)

    iou_orig = iou(pred, gt)
    iou_flip = iou(1 - pred, gt)
    return pred if iou_orig >= iou_flip else (1 - pred)

## Funções de Segmentação:

### Fuzzy C-Means

In [54]:
def cmeans_labels(
    features: np.ndarray,
    n_clusters: int = 2,
    m: float = 2,
    error: float = 1e-5,
    maxiter: int = 1000,
    seed: int = 0,
) -> Tuple[np.ndarray, np.ndarray]:
    """Aplica o Fuzzy C‑Means e retorna rótulos e centros.

    O input ``features`` pode ser uma imagem 2‑D ou uma matriz (H, W, K). Caso
    possua duas dimensões, ela é expandida para possuir um canal adicional.

    Parameters
    ----------
    features : np.ndarray
        Array de características com dimensão (H, W) ou (H, W, K), normalizado
        em [0,1].
    n_clusters : int, optional
        Número de clusters desejados, por padrão 3.
    m : float, optional
        Parâmetro de fuzzificação (m > 1). Valores maiores resultam em
        partições mais difusas. Padrão: 1.6.
    error : float, optional
        Critério de parada. A iteração termina quando a diferença entre
        partições consecutivas for menor que ``error``. Padrão: 1e‑5.
    maxiter : int, optional
        Número máximo de iterações do algoritmo. Padrão: 1000.
    seed : int, optional
        Semente do gerador de números aleatórios. Padrão: 0.

    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        ``labels`` : array 2‑D de inteiros com tamanho (H, W)
            Rótulos de cluster atribuídos a cada pixel.
        ``centers`` : array 2‑D com tamanho (n_clusters, K)
            Centróides no espaço de características.
    """
    # Garante que features tenha terceira dimensão (canal)
    if features.ndim == 2:
        features = features[..., None]
    H, W, K = features.shape
    # Transforma features em forma (N, K) para cálculos
    X_flat = features.reshape(-1, K)  # shape (N, K)
    N = X_flat.shape[0]
    c = n_clusters
    rng = default_rng(seed)
    # Inicializa a matriz de pertinência U de forma aleatória
    U = rng.random((c, N))
    U /= U.sum(axis=0, keepdims=True)
    # Iterações do algoritmo FCM
    for _ in range(maxiter):
        U_old = U.copy()
        # Calcula os centros (c, K): soma((u_ik)^m * x_k) / soma((u_ik)^m)
        um = U ** m
        centers = um @ X_flat / um.sum(axis=1, keepdims=True)
        # Distâncias dos centros aos pontos (c, N)
        diff = X_flat[None, :, :] - centers[:, None, :]
        dist = np.linalg.norm(diff, axis=2) + 1e-10  # evita divisão por zero
        # Atualiza U (c, N)
        power = 2.0 / (m - 1.0)
        for i in range(c):
            ratio = (dist[i] / dist) ** power  # shape (c, N)
            denom = ratio.sum(axis=0)  # shape (N,)
            U[i] = 1.0 / denom
        # Verifica convergência
        if np.max(np.abs(U - U_old)) < error:
            break
    labels = np.argmax(U, axis=0).reshape(H, W)
    return labels, centers

### K-Means:

In [55]:
def kmeans_labels(
    features: np.ndarray,
    n_clusters: int = 3,
    seed: int = 0,
    n_init: int = 10,
    max_iter: int = 300,
) -> Tuple[np.ndarray, np.ndarray]:
    """Aplica K‑Means (scikit‑learn) e retorna rótulos e centros.

    Parameters
    ----------
    features : np.ndarray
        Imagem ou matriz de características com dimensão (H, W) ou (H, W, K).
    n_clusters : int, optional
        Número de clusters desejados. Padrão: 3.
    seed : int, optional
        Semente do gerador de números aleatórios. Padrão: 0.
    n_init : int, optional
        Número de inicializações diferentes. O K‑Means será executado ``n_init``
        vezes com diferentes centroides aleatórios e o melhor resultado será
        escolhido (menor inércia). Padrão: 10.
    max_iter : int, optional
        Número máximo de iterações do algoritmo para uma única execução. Padrão:
        300.

    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        ``labels`` : array 2‑D de inteiros com tamanho (H, W)
            Rótulos de cluster atribuídos a cada pixel.
        ``centers`` : array 2‑D com tamanho (n_clusters, K)
            Centróides no espaço de características.
    """
    if features.ndim == 2:
        features = features[..., None]
    H, W, K = features.shape
    X = features.reshape(-1, K)  # shape (N, K)
    kmeans = KMeans(
        n_clusters=n_clusters,
        random_state=seed,
        n_init=n_init,
        max_iter=max_iter,
    )
    labels_flat = kmeans.fit_predict(X)
    centers = kmeans.cluster_centers_
    labels = labels_flat.reshape(H, W)
    return labels, centers

### QFFCM

In [ ]:
def cmeans_labels(
    features: np.ndarray,
    n_clusters: int = 2,
    m: float = 2.0,         # m = 2
    error: float = 1e-3,    # error = 1e-2
    maxiter: int = 20,
    nbins: int = 8,
    seed: int = 0,
    lambda_s: float = 50.0,
    mu_s: float = 10.0,
    alpha: float = 0.1,     # alpha = 0.1
    rho_initial: float = 20.0,  # rho = 20
    beta_rho: float = 2.0,      # beta_rho = 2
    admm_inner_maxiter: int = 3,  # admm_inner_maxiter = 3
) -> Tuple[np.ndarray, np.ndarray]:
    """
    QFFCM = Fast FCM + 3-ADMM-H + QAOA
    """
    if features.ndim != 2:
        raise ValueError(
            f"QFFCM implementado aqui apenas para imagens 2D (H,W); recebi ndim={features.ndim}."
        )

    H, W = features.shape
    vals = features.astype(np.float64).ravel()  # (N,)

    vmin = float(vals.min())
    vmax = float(vals.max()) + 1e-12

    hist, bin_edges = np.histogram(vals, bins=nbins, range=(vmin, vmax)) # Reduzindo de N pixels para L bins
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])  # (L,)
    weights = hist.astype(np.float64)                     # (L,)
    L = bin_centers.shape[0]

    if np.all(weights == 0):
        labels = np.zeros((H, W), dtype=int)
        centers = np.array([[vmin]], dtype=np.float64)
        return labels, centers

    rng = default_rng(seed)
    C = n_clusters

    nonzero_bins = np.where(weights > 0)[0] # Inicialização dos clusters
    if len(nonzero_bins) >= C:
        init_idx = rng.choice(nonzero_bins, size=C, replace=False)
    else:
        init_idx = rng.choice(L, size=C, replace=False)

    V = bin_centers[init_idx].reshape(C, 1)   # (C,1)

    sampler = StatevectorSampler(seed=seed)
    inner_qaoa = QAOA(
        sampler=sampler,
        reps=3,                       # reps = 3 (profundidade QAOA)
        optimizer=COBYLA(maxiter=10),
    )
    qubo_optimizer = MinimumEigenOptimizer(inner_qaoa)
    cont_optimizer = CobylaOptimizer()

    admm_params = ADMMParameters(
        rho_initial=rho_initial,
        beta=beta_rho,
        factor_c=10.0,
        maxiter=admm_inner_maxiter,
        three_block=True,
        vary_rho=UPDATE_RHO_BY_RESIDUALS,
        tau_incr=2.0,
        tau_decr=2.0,
        mu_res=10.0,
    )
    admm = ADMMOptimizer(
        qubo_optimizer=qubo_optimizer,
        continuous_optimizer=cont_optimizer,
        params=admm_params,
    )

    # helper interna: constrói QP/QUBO no espaço dos bins
    def build_ffcm_admm_qp_hist(
        bin_centers: np.ndarray,
        V: np.ndarray,
    ) -> QuadraticProgram:
        """
        Constrói o QuadraticProgram para FFCM + MBO binário nos bins.
        """
        X_hist = bin_centers.reshape(-1, 1)  # (L,1)
        L_loc, K = X_hist.shape             # K=1
        C_loc = V.shape[0]

        D = np.sum((X_hist[:, None, :] - V[None, :, :])**2, axis=2)  # (L_loc,C_loc)

        qp = QuadraticProgram("ffcm_mbo_hist")

        for i in range(L_loc):
            for k in range(C_loc):
                qp.continuous_var(name=f"u_{i}_{k}", lowerbound=0.0, upperbound=1.0)
                qp.binary_var(name=f"z_{i}_{k}")

        lin: Dict[str, float] = {}
        quad: Dict[tuple, float] = {}

        def add_lin(var: str, coeff: float):
            if abs(coeff) < 1e-15:
                return
            lin[var] = lin.get(var, 0.0) + float(coeff)

        def add_quad(v1: str, v2: str, coeff: float):
            if abs(coeff) < 1e-15:
                return
            key = (v1, v2)
            quad[key] = quad.get(key, 0.0) + float(coeff)

        # (1) termo FFCM clássico: sum_i,k w_i * d_{ik} * u_{ik}^2
        for i in range(L_loc):
            for k in range(C_loc):
                ui = f"u_{i}_{k}"
                add_quad(ui, ui, weights[i] * D[i, k])

        # (2) penalização de fuzzy-sum: lambda_s * (sum_k u_{ik} - 1)^2
        for i in range(L_loc):
            for k in range(C_loc):
                uk = f"u_{i}_{k}"
                add_quad(uk, uk, lambda_s)
                for ell in range(k + 1, C_loc):
                    ul = f"u_{i}_{ell}"
                    add_quad(uk, ul, 2.0 * lambda_s)
            for k in range(C_loc):
                uk = f"u_{i}_{k}"
                add_lin(uk, -2.0 * lambda_s)

        # (3) penalização one-hot binária: mu_s * (sum_k z_{ik} - 1)^2
        for i in range(L_loc):
            for k in range(C_loc):
                zk = f"z_{i}_{k}"
                add_quad(zk, zk, mu_s)
                for ell in range(k + 1, C_loc):
                    zl = f"z_{i}_{ell}"
                    add_quad(zk, zl, 2.0 * mu_s)
            for k in range(C_loc):
                zk = f"z_{i}_{k}"
                add_lin(zk, -2.0 * mu_s)

        # (4) custo linear em z
        for i in range(L_loc):
            for k in range(C_loc):
                zk = f"z_{i}_{k}"
                add_lin(zk, alpha * weights[i] * D[i, k])

        qp.minimize(linear=lin, quadratic=quad)
        return qp

    # ----------------------------------------------------------------------
    # 4) Laço externo: QFFCM + 3-ADMM-H
    # ----------------------------------------------------------------------
    U = np.zeros((L, C))
    Z = np.zeros((L, C))
    rho = admm_params.rho_initial

    for it in range(maxiter):
        U_old = U.copy()

        qp = build_ffcm_admm_qp_hist(bin_centers, V)
        print("n_binárias (qubits no QAOA):", qp.get_num_binary_vars())

        result = admm.solve(qp)

        for i in range(L):
            for k in range(C):
                U[i, k] = result.variables_dict[f"u_{i}_{k}"]
                Z[i, k] = result.variables_dict[f"z_{i}_{k}"]

        # atualização de V (FFCM ponderado)
        um = U ** m
        w_um = um * weights[:, None]

        num = (w_um.T @ bin_centers.reshape(-1, 1))
        den = w_um.sum(axis=0).reshape(C, 1) + 1e-12
        V = num / den

        if np.max(np.abs(U - U_old)) < error:
            break

        rho *= beta_rho

    # ----------------------------------------------------------------------
    # 5) Rótulos finais em bins e pixels
    # ----------------------------------------------------------------------
    labels_bins = np.argmax(U, axis=1)  # (L,)

    bin_idx = np.digitize(vals, bin_edges[:-1], right=False)
    bin_idx = np.clip(bin_idx, 0, L - 1)

    labels_flat = labels_bins[bin_idx]
    labels = labels_flat.reshape(H, W)

    centers = V.copy()

    return labels, centers

### Funções de Avaliação de Qualidade:

#### Métricas Internas:

In [56]:
def dunn_index(
    features: np.ndarray,
    labels: np.ndarray,
) -> float:
    """Calcula uma versão aproximada do índice de Dunn para clusters grandes.

    O índice de Dunn original é definido como a razão entre a menor
    distância inter‑cluster e o maior diâmetro intra‑cluster【699562914633650†L170-L189】. Ele avalia o quão
    separados e compactos estão os clusters: valores maiores indicam melhor
    separação entre clusters e menor dispersão dentro dos clusters.

    Nesta implementação, para tornar o cálculo viável em imagens com dezenas
    de milhares de pontos, a distância inter‑cluster é calculada utilizando os
    centróides dos clusters (distância euclidiana mínima entre centros), e o
    diâmetro intra‑cluster é aproximado como o dobro da maior distância entre
    qualquer ponto do cluster e seu centroide.

    Parameters
    ----------
    features : np.ndarray
        Matriz de características de forma (H, W, K) ou (N, K).
    labels : np.ndarray
        Array de rótulos de clusters (H, W) ou (N,) correspondente aos pontos em
        ``features``.

    Returns
    -------
    float
        Valor aproximado do índice de Dunn. Quanto maior, melhor a
        segmentação.
    """
    # Ajusta dimensões para formato (N, K)
    if features.ndim == 3:
        X = features.reshape(-1, features.shape[-1])
    else:
        X = features.reshape(-1, 1)
    labels_flat = labels.reshape(-1)
    unique_labels = np.unique(labels_flat)
    # Calcula centróides
    centers = np.stack([
        X[labels_flat == lbl].mean(axis=0) if np.any(labels_flat == lbl) else np.zeros(X.shape[1])
        for lbl in unique_labels
    ])
    # Calcula matriz de distâncias entre centróides
    dist_centers = pairwise_distances(centers)
    # Ignora a diagonal ao buscar o mínimo valor não nulo
    np.fill_diagonal(dist_centers, np.inf)
    min_inter_cluster = dist_centers.min()
    # Calcula diâmetro aproximado de cada cluster: 2 * max(distância ao centro)
    max_diameter = 0.0
    for idx, lbl in enumerate(unique_labels):
        points = X[labels_flat == lbl]
        if points.shape[0] <= 1:
            continue
        center = centers[idx]
        distances = np.linalg.norm(points - center, axis=1)
        diameter = 2.0 * distances.max()
        if diameter > max_diameter:
            max_diameter = diameter
    if max_diameter == 0:
        return np.inf
    return float(min_inter_cluster / max_diameter)

#### Aplicando a Métrica de Dunn e Algumas outras:

In [57]:
def evaluate_clustering(
    features: np.ndarray,
    labels: np.ndarray,
    method_name: str,
) -> Dict[str, float]:
    """Calcula índices internos de validação de cluster para uma segmentação.

    Parameters
    ----------
    features : np.ndarray
        Matriz de características (H, W, K) ou (H, W).
    labels : np.ndarray
        Rótulos obtidos para cada ponto (H, W).
    method_name : str
        Nome do método (para fins informativos). Não interfere no cálculo.

    Returns
    -------
    Dict[str, float]
        Dicionário contendo as chaves ``'DB'`` (Davies–Bouldin), ``'DI'``
        (Dunn) e ``'CH'`` (Calinski–Harabasz).
    """
    # Ajusta as dimensões do conjunto de pontos
    if features.ndim == 2:
        X = features[..., None].reshape(-1, 1)
    else:
        X = features.reshape(-1, features.shape[-1])
    y = labels.reshape(-1)
    # Davies–Bouldin: média de similaridade de cada cluster com seu cluster mais próximo; valores menores indicam clusters mais compactos【518966911363608†L676-L683】
    db = davies_bouldin_score(X, y)
    # Calinski–Harabasz: razão entre dispersão inter‑cluster e intra‑cluster
    ch = calinski_harabasz_score(X, y)
    # Dunn index (aproximado): quanto maior, melhor【699562914633650†L170-L189】
    di = dunn_index(features, labels)
    return {"DB": db, "CH": ch, "DI": di}

## Funções Auxiliares(executa a segmentação e mede o desempenho)

In [58]:
def run_fcm(
    image: np.ndarray,
    n_clusters: int = 2,
    seed: int = 0,
    m: float = 2.0,
    error: float = 1e-3,
    maxiter: int = 1000,
    nbins: int = 8,
) -> Dict[str, Any]:
    """Executa segmentação Fuzzy C‑Means e retorna resultados.

    Esta função encapsula a chamada ao algoritmo FCM, calcula o tempo de
    processamento e avalia a segmentação utilizando ``evaluate_clustering``.
    O retorno é um dicionário com campos que podem ser manipulados
    independentemente em outros contextos.

    Parameters
    ----------
    image : np.ndarray
        Imagem 2‑D normalizada em ``[0,1]``.
    n_clusters : int, optional
        Número de clusters para segmentação. Padrão: 3.
    seed : int, optional
        Semente aleatória para reproducibilidade. Padrão: 0.
    m : float, optional
        Parâmetro de fuzzificação para o FCM. Padrão: 1.6.
    error : float, optional
        Critério de parada para o FCM. Padrão: 1e‑5.
    maxiter : int, optional
        Número máximo de iterações para o FCM. Padrão: 1000.

    Returns
    -------
    Dict[str, Any]
        Dicionário com as chaves ``'labels'``, ``'centers'``, ``'metrics'`` e
        ``'time'``. O campo ``'labels'`` contém a máscara de rótulos (H, W).
    """
    t0 = time.time()
    labels, centers = cmeans_labels(
        image,
        n_clusters=n_clusters,
        m=m,
        error=error,
        maxiter=maxiter,
        seed=seed,
        nbins=nbins,
    )
    elapsed = time.time() - t0
    metrics = evaluate_clustering(image, labels, method_name="FCM")
    return {
        "labels": labels,
        "centers": centers,
        "metrics": metrics,
        "time": elapsed,
    }



def run_kmeans(
    image: np.ndarray,
    n_clusters: int = 3,
    seed: int = 0,
    n_init: int = 10,
    max_iter: int = 300,
) -> Dict[str, Any]:
    """Executa segmentação K‑Means e retorna resultados.

    Parameters
    ----------
    image : np.ndarray
        Imagem 2‑D normalizada em ``[0,1]``.
    n_clusters : int, optional
        Número de clusters para segmentação. Padrão: 3.
    seed : int, optional
        Semente aleatória para reproducibilidade. Padrão: 0.
    n_init : int, optional
        Número de inicializações para o K‑Means. Padrão: 10.
    max_iter : int, optional
        Número máximo de iterações para o K‑Means. Padrão: 300.

    Returns
    -------
    Dict[str, Any]
        Dicionário com as chaves ``'labels'``, ``'centers'``, ``'metrics'`` e
        ``'time'``. O campo ``'labels'`` contém a máscara de rótulos (H, W).
    """
    t0 = time.time()
    labels, centers = kmeans_labels(
        image,
        n_clusters=n_clusters,
        seed=seed,
        n_init=n_init,
        max_iter=max_iter,
    )
    elapsed = time.time() - t0
    metrics = evaluate_clustering(image, labels, method_name="KMeans")
    return {
        "labels": labels,
        "centers": centers,
        "metrics": metrics,
        "time": elapsed,
    }

## Plote das Imagens segmentadas pelo dois métodos: 

In [59]:
def plot_comparison(
    labels_fcm: np.ndarray,
    labels_km: np.ndarray,
    title_base: str,
    out_dir: Path,
    cmap: str = "viridis",
) -> Path:
    """Plota lado a lado as máscaras de segmentação de FCM e K‑Means.

    Parameters
    ----------
    labels_fcm : np.ndarray
        Máscara de rótulos obtida com Fuzzy C‑Means (H, W).
    labels_km : np.ndarray
        Máscara de rótulos obtida com K‑Means (H, W).
    title_base : str
        Base para o título e nome do arquivo (por exemplo, nome da imagem).
    out_dir : Path
        Diretório onde a figura será salva.
    cmap : str, optional
        Paleta de cores utilizada para exibir as máscaras. Padrão: ``'viridis'``.

    Returns
    -------
    Path
        Caminho do arquivo de imagem salvo.
    """
    out_dir.mkdir(parents=True, exist_ok=True)
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    # FCM
    axes[0].imshow(labels_fcm, cmap=cmap)
    axes[0].set_title(f"{title_base} Index - FCM Segmentation", fontsize=12)
    axes[0].axis("off")
    # K‑Means
    axes[1].imshow(labels_km, cmap=cmap)
    axes[1].set_title(f"{title_base} Index - K-Means Segmentation", fontsize=12)
    axes[1].axis("off")
    # Título geral
    fig.suptitle(
        "Segmentation Comparison Between Fuzzy C-Means and K-Means",
        fontsize=14,
        y=1.02,
        fontweight="bold",
    )
    plt.tight_layout()
    out_path = out_dir / f"{title_base}_comparacao.png"
    plt.savefig(out_path, bbox_inches="tight")
    plt.close(fig)
    return out_path

## Função principal (roda tudo anterior para uma imagem):

In [60]:
def segment_image(
    image: np.ndarray,
    image_name: str,
    n_clusters: int = 2,
    out_dir: Path | None = None,
    seed: int = 0,
    fcm_params: Dict[str, Any] | None = None,
    kmeans_params: Dict[str, Any] | None = None,
    normalize: Optional[str] = None,              # <- NOVO
    norm_params: Optional[Dict[str, float]] = None,  # <- NOVO (para globais)
) -> Dict[str, Any]:

    # ... dentro da função, antes de chamar run_fcm/run_kmeans:
    img_in, used_norm = apply_normalization(image, normalize=normalize, params=norm_params)

    fcm_res = run_fcm(
        image=img_in,
        n_clusters=n_clusters,
        seed=seed,
        **(fcm_params or {}),
    )
    km_res = run_kmeans(
        image=img_in,
        n_clusters=n_clusters,
        seed=seed,
        **(kmeans_params or {}),
    )

       # Gera figura comparativa, se desejado.
    plot_path = None
    if out_dir is not None:
        out_dir = Path(out_dir)
        plot_path = plot_comparison(
            labels_fcm=fcm_res["labels"],
            labels_km=km_res["labels"],
            title_base=image_name,
            out_dir=out_dir,
        )

    return {
        "fcm": fcm_res,
        "kmeans": km_res,
        "plot_path": plot_path,
        "normalization": used_norm,
    }

### Roda tudo para multiplas Imagens:

In [61]:
def run_batch(
    image_paths: List[Path],
    n_clusters: int = 2,
    seed: int = 0,
    fcm_params: Dict[str, Any] | None = None,
    kmeans_params: Dict[str, Any] | None = None,
    out_dir: Path | None = None,
    normalize: Optional[str] = None,                 # <- NOVO
    norm_params: Optional[Dict[str, float]] = None,  # <- NOVO
) -> List[Tuple[str, Dict[str, Any]]]:
    """Processa uma lista de imagens, aplicando segmentação e retornando resultados.

        Parameters
        ----------
        image_paths : list of Path
            Lista de caminhos das imagens a serem segmentadas.
        n_clusters : int, optional
            Número de clusters para segmentação. Padrão: 3.
        seed : int, optional
            Semente aleatória para reproducibilidade. Padrão: 0.
        fcm_params : dict, optional
            Parâmetros adicionais para ``run_fcm``.
        kmeans_params : dict, optional
            Parâmetros adicionais para ``run_kmeans``.
        out_dir : Path, optional
            Diretório onde as figuras de comparação serão salvas. Se ``None``,
            figuras não serão geradas.

        Returns
        -------
        List[Tuple[str, Dict[str, Any]]]
            Lista de tuplas com o nome da imagem e o dicionário de resultados
            retornado por ``segment_image``.
        """
    results = []
    for path in image_paths:
        img = np.asarray(Image.open(path).convert("L"), dtype=np.float32)  # sem normalizar
        name = path.stem
        res = segment_image(
            image=img,
            image_name=name,
            n_clusters=n_clusters,
            out_dir=out_dir,
            seed=seed,
            fcm_params=fcm_params,
            kmeans_params=kmeans_params,
            normalize=normalize,          # <- NOVO
            norm_params=norm_params,      # <- NOVO
        )
        results.append((name, res))
    return results

def run_batch_arrays(
    images: List[np.ndarray],
    names: List[str] | None = None,
    n_clusters: int = 3,
    seed: int = 0,
    fcm_params: Dict[str, Any] | None = None,
    kmeans_params: Dict[str, Any] | None = None,
    out_dir: Path | None = None,
    normalize: Optional[str] = None,
    norm_params: Optional[Dict[str, float]] = None,
) -> List[Tuple[str, Dict[str, Any]]]:
    if names is None:
        names = [f"img_{i}" for i in range(len(images))]

    results: List[Tuple[str, Dict[str, Any]]] = []
    for name, img in zip(names, images):
        res = segment_image(
            image=img.astype(np.float32),
            image_name=name,
            n_clusters=n_clusters,
            out_dir=out_dir,
            seed=seed,
            fcm_params=fcm_params,
            kmeans_params=kmeans_params,
            normalize=normalize,
            norm_params=norm_params,
        )
        results.append((name, res))
    return results


### Função que mostra os Resultados:

In [62]:


def print_results(results: List[Tuple[str, Dict[str, Any]]]) -> None:
    """Imprime uma tabela comparativa de índices e tempos de segmentações.

    Esta função recebe a lista retornada por ``run_batch`` e exibe uma tabela
    formatada no console com os valores de Davies–Bouldin, Dunn, Calinski–Harabasz
    e tempo de processamento para cada método e cada imagem.

    Parameters
    ----------
    results : list of (str, dict)
        Lista de tuplas com o nome da imagem e o dicionário de resultados.
    """
    print("\nTabela comparativa de índices e tempos:\n")
    header = f"{'Imagem':<15} | {'Método':<7} | {'DB':>10} | {'DI':>10} | {'CH':>10} | {'Tempo (s)':>10}"
    print(header)
    print("-" * len(header))
    for name, res in results:
        # FCM
        fm = res["fcm"]["metrics"]
        print(
            f"{name:<15} | {'FCM':<7} | {fm['DB']:>10.4f} | {fm['DI']:>10.4f} | {fm['CH']:>10.4f} | {res['fcm']['time']:>10.4f}"
        )
        # K‑Means
        km = res["kmeans"]["metrics"]
        print(
            f"{name:<15} | {'KMeans':<7} | {km['DB']:>10.4f} | {km['DI']:>10.4f} | {km['CH']:>10.4f} | {res['kmeans']['time']:>10.4f}"
        )
        print()

# Metricas Externas de Comparação(Usando GroundTruth)

In [63]:
def hausdorff_distance(ground: np.ndarray, seg: np.ndarray) -> float:
    """Calcula a distância de Hausdorff (bidirecional) entre duas máscaras binárias.

    A métrica considera a maior distância entre os pixels do conjunto A até o pixel mais próximo
    no conjunto B e vice‑versa. Quando uma das máscaras está vazia (sem pixels verdadeiros),
    retorna ``np.inf`` indicando que a distância é indefinida.

    Parameters
    ----------
    ground : np.ndarray
        Máscara de referência (ground truth). Valores não‑zero são considerados positivos.
    seg : np.ndarray
        Máscara de segmentação obtida. Valores não‑zero são considerados positivos.

    Returns
    -------
    float
        Distância de Hausdorff entre as duas máscaras.
    """
    # Converte as entradas em booleano (True para pixel de interesse)
    a = ground.astype(bool)
    b = seg.astype(bool)
    if a.shape != b.shape:
        raise ValueError("As dimensões de 'ground' e 'seg' devem ser iguais para o cálculo da Hausdorff.")
    # Se um dos conjuntos estiver vazio, a distância é infinita (não há comparação possível)
    if not np.any(a) or not np.any(b):
        return float('inf')
    # Distância transform da máscara negativa de b: para cada pixel, distância até o pixel mais próximo em b
    dt_b = distance_transform_edt(~b)
    # Distância de cada ponto em a até o conjunto b
    dist_a_to_b = dt_b[a]
    max_a_to_b = dist_a_to_b.max() if dist_a_to_b.size > 0 else 0.0
    # Distância transform de a
    dt_a = distance_transform_edt(~a)
    dist_b_to_a = dt_a[b]
    max_b_to_a = dist_b_to_a.max() if dist_b_to_a.size > 0 else 0.0
    return float(max(max_a_to_b, max_b_to_a))


def dice_coefficient(ground: np.ndarray, seg: np.ndarray) -> float:
    """Calcula o coeficiente de Dice entre duas máscaras binárias.

    O coeficiente de Dice (F1 score para conjuntos) é definido como:

    .. math:: 	ext{Dice}(A, B) = rac{2|A \cap B|}{|A| + |B|}

    onde |A| e |B| são os números de pixels positivos em cada máscara.

    Retorna 1.0 quando ambos os conjuntos são vazios.
    """
    a = ground.astype(bool)
    b = seg.astype(bool)
    intersection = np.logical_and(a, b).sum()
    size_a = a.sum()
    size_b = b.sum()
    # Se ambas as máscaras forem vazias, retorna 1 (acordo perfeito)
    if size_a + size_b == 0:
        return 1.0
    return 2.0 * intersection / float(size_a + size_b)


def iou_coefficient(ground: np.ndarray, seg: np.ndarray) -> float:
    """Calcula o coeficiente Intersection over Union (IoU) ou índice de Jaccard.

    O IoU é definido como:

    .. math:: 	ext{IoU}(A, B) = rac{|A \cap B|}{|A \cup B|}

    onde |A \cup B| é o número de pixels que pertencem a pelo menos uma das máscaras.

    Retorna 1.0 quando ambos os conjuntos são vazios.
    """
    a = ground.astype(bool)
    b = seg.astype(bool)
    intersection = np.logical_and(a, b).sum()
    union = np.logical_or(a, b).sum()
    if union == 0:
        return 1.0
    return intersection / float(union)


def validacao_metricas_externas(ground: np.ndarray, segmentado: np.ndarray) -> dict:
    """Calcula métricas externas de avaliação de segmentação.

    Esta função consolida três métricas comumente utilizadas para avaliar a
    concordância entre uma segmentação e a segmentação de referência:

    * "Hausdorff": distância de Hausdorff bidirecional entre as máscaras;
    * "Dice": coeficiente de Dice (também conhecido como F1 score em segmentação);
    * "IoU": Intersection over Union (índice de Jaccard).

    Parameters
    ----------
    ground : np.ndarray
        Máscara de referência (ground truth). Qualquer valor não‑zero será interpretado como pixel positivo.
    segmentado : np.ndarray
        Máscara segmentada a ser validada. Valores não‑zero serão interpretados como pixel positivo.

    Returns
    -------
    dict
        Dicionário com as métricas calculadas, contendo as chaves "Hausdorff", "Dice" e "IoU".
    """
    return {
        'Hausdorff': hausdorff_distance(ground, segmentado),
        'Dice': dice_coefficient(ground, segmentado),
        'IoU': iou_coefficient(ground, segmentado),
    }

# Importando dados e Testando local:

In [64]:
import numpy as np

linha = 0

data = np.load(f"Imagens_e_indices/Image[{linha}]/Image[{linha}]indices_diff.npz")

img_diff_ndvi = data["diff_ndvi"].astype(np.float32)
img_diff_nbr = data["diff_nbr"].astype(np.float32)
img_diff_nbrswir = data["diff_nbrswir"].astype(np.float32)

imagens = [
    img_diff_nbr,
    img_diff_ndvi,
    img_diff_nbrswir]

fcm_params = {
    "m": 2.0,        # grau de fuzzificação
    "maxiter": 500,  # número máximo de iterações
}


res = segment_image(
    image=img_diff_nbrswir,
    image_name="imagem_NBRswir",
    n_clusters=2,
    out_dir=None,
    seed=0,
    fcm_params=fcm_params,
    kmeans_params=None,
    normalize=None,
    norm_params=None
)

# Salvando todas as imagens e tabelas de métricas

In [ ]:
def process_all_images(
    root_dir: Path,
    out_root: Path,
    n_clusters: int = 2,
    seed: int = 0,
    normalize: Optional[str] = "minmax",
) -> List[Dict[str, Any]]:
    out_root.mkdir(parents=True, exist_ok=True)
    metric_rows: List[Dict[str, Any]] = []

    image_dirs = sorted(
        p for p in root_dir.iterdir()
        if p.is_dir() and p.name.startswith("Image")
    )

    if not image_dirs:
        print(f"Nenhuma pasta 'Image[...]' encontrada em {root_dir}")
        return metric_rows

    for img_dir in image_dirs:
        img_id = img_dir.name
        print(f"\n=== Processando {img_id} ===")

        gt_candidates = sorted(img_dir.glob("*GroundTruth*.npz"))
        idx_candidates = sorted(img_dir.glob("*indices*.npz"))

        if not gt_candidates or not idx_candidates:
            print(f"  [AVISO] Pulando {img_id}: não achei GroundTruth/indices_diff .npz")
            continue

        gt_path = gt_candidates[0]
        idx_path = idx_candidates[0]

        print(f"  Ground truth : {gt_path.name}")
        print(f"  Índices      : {idx_path.name}")

        gt_npz = np.load(gt_path)
        gt_key = gt_npz.files[0]
        gt_array = gt_npz[gt_key]
        ground_bin = (gt_array > 0).astype(np.uint8)

        indices_npz = np.load(idx_path)

        for index_name in indices_npz.files:
            idx_img = indices_npz[index_name].astype(np.float32)

            if idx_img.shape != ground_bin.shape:
                print(
                    f"  [AVISO] Índice '{index_name}' shape {idx_img.shape} "
                    f"diferente da GT {ground_bin.shape}. Pulando."
                )
                continue

            print(f"  -> Índice: {index_name} (shape={idx_img.shape})")

            full_name = f"{img_id}_{index_name}"
            pair_out_dir = out_root / img_id / index_name

            fcm_params = {
                "m": 2.0,
                "error": 1e-3,   # error = 1e-3
                "maxiter": 15,
                "nbins": 8,
            }
            kmeans_params = {
                "n_init": 10,
                "max_iter": 300,
            }

            res = segment_image(
                image=idx_img,
                image_name=full_name,
                n_clusters=n_clusters,
                out_dir=pair_out_dir,
                seed=seed,
                fcm_params=fcm_params,
                kmeans_params=kmeans_params,
                normalize=normalize,
                norm_params=None,
            )
            
            print(f"  [OK] Segmentação QFFCM + KMeans concluída para {full_name}", flush=True)

            fcm_labels = res["fcm"]["labels"]
            km_labels = res["kmeans"]["labels"]

            fcm_bin = (fcm_labels != 0).astype(np.uint8)
            km_bin = (km_labels != 0).astype(np.uint8)

            fcm_bin = maybe_flip_binary(fcm_bin, ground_bin)
            km_bin = maybe_flip_binary(km_bin, ground_bin)

            ext_fcm = validacao_metricas_externas(ground_bin, fcm_bin)
            ext_km = validacao_metricas_externas(ground_bin, km_bin)

            int_fcm = res["fcm"]["metrics"]
            int_km = res["kmeans"]["metrics"]

            metric_rows.append({
                "image": img_id,
                "index": index_name,
                "method": "QFFCM",
                "DB": int_fcm["DB"],
                "DI": int_fcm["DI"],
                "CH": int_fcm["CH"],
                "Hausdorff": ext_fcm["Hausdorff"],
                "Dice": ext_fcm["Dice"],
                "IoU": ext_fcm["IoU"],
                "time_seconds": res["fcm"]["time"],
            })

            metric_rows.append({
                "image": img_id,
                "index": index_name,
                "method": "KMeans",
                "DB": int_km["DB"],
                "DI": int_km["DI"],
                "CH": int_km["CH"],
                "Hausdorff": ext_km["Hausdorff"],
                "Dice": ext_km["Dice"],
                "IoU": ext_km["IoU"],
                "time_seconds": res["kmeans"]["time"],
            })

    return metric_rows

def salvar_tabela_metricas(metric_rows: List[Dict[str, Any]], out_root: Path) -> None:
    if not metric_rows:
        print("Nenhuma métrica para salvar.")
        return

    out_root.mkdir(parents=True, exist_ok=True)
    csv_path = out_root / "metricas_segmentacao.csv"

    try:
        import pandas as pd
    except ImportError:
        keys = list(metric_rows[0].keys())
        with csv_path.open("w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=keys)
            writer.writeheader()
            writer.writerows(metric_rows)
        print(f"Tabela de métricas salva em (sem pandas): {csv_path}")
    else:
        df = pd.DataFrame(metric_rows)
        df.to_csv(csv_path, index=False)
        print(f"Tabela de métricas salva em: {csv_path}")

### Main

In [ ]:
if __name__ == "__main__":
    base_dir = Path("Imagens_e_indices")
    resultados_dir = Path("Resultados_segmentacao")

    metric_rows = process_all_images(
        root_dir=base_dir,
        out_root=resultados_dir,
        n_clusters=2,
        seed=0,
        normalize="minmax"
    )

    salvar_tabela_metricas(metric_rows, resultados_dir)
    print("\nProcessamento concluído.")